In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
CURRENT_PE = {6: [5.96, 6.33, 6.32, 6.30, 6.27],
             5: [4.98, 4.97, 4.95, 4.96],
             4: [3.99, 3.98, 4.0],
             3: [2.99, 2.97, 2.98]}
CURRENT_PA = {6: [5.84, 6.18, 6.17, 6.16, 6.15, 6.13],
             5: [4.88, 4.93, 4.94, 4.95],
             4: [3.97, 3.98, 3.99],
             3: [2.96, 2.97, 2.98, 2.96, 2.97]}

In [3]:
class Spectrum:
    global CURRENT_PA, CURRENT_PE
    def __init__(self, name, element = "Neon", directory = "raw/"):
        
        self.name = name
        if directory in name:
            self.name = name[len(directory):]          
        self.element = element
        if "calibracion" in name:
            self.element = "Calibration"
        self.current = 0
        self.time = 15
        self.data = fits.getdata("%s%s"%(directory, self.name))
        self.get_current()
        self.normalize()
        
    def get_current(self):
        try:
            intensity = int(self.name[2])
        except:
            pass
        if "pa" in self.name:
            data = CURRENT_PA[intensity]
            mean = np.array(data).mean()
            self.current = mean
            self.type = "Parallel"
        elif 'pe' in self.name:
            data = CURRENT_PE[intensity]
            mean = np.array(data).mean()
            self.current = mean
            self.type = "Perpendicular"
        else:
            self.type = None
            
    def normalize(self):
        mean = np.mean(self.data)
        min_ = np.min(self.data)
        thresh = mean + min_
        pos = np.where(self.data > thresh)
        self.data[pos] = thresh
        self.data = (self.data - min_)/(thresh - min_)
        
    def make_slice(self, ys, xs):
        if ys == None:
            return self.data
        center, width = ys
        half = int(width/2)
        data = self.data[center-half:center+half]
        if xs == None:
            return data
        xlow, xhigh = xs
        return data[:, xlow:xhigh]
            
        
    def plot(self, ys, xs):
        fig, (ax1, ax2) = plt.subplots(2, sharex=True)
        data = self.make_slice(ys, xs)
        complete = np.sum(data, axis=0)
#        integral = np.trapz(complete)
        integral = complete#/integral
        ax1.imshow(self.data, cmap='gray')
        ax1.axis('off')
        if xs == None:
            ax2.plot(integral)
            ax2.set_xlim(0, data.shape[1])
        if xs != None:
            ax2.plot(np.arange(xs[0], xs[1]), integral)
            ax2.set_xlim(xs[0], xs[1])
        fig.subplots_adjust(hspace=0)
        title = self.element
        if self.element != "Calibration":
            if self.type != None:
                title = r"%s %s $I\approx %.2f$A"%(self.element, self.type, self.current)
            else:
                title = r"%s with out $\vec{B}$"%(self.element)
        ax1.set_title(title)
        return fig

In [4]:
calibration = Spectrum('raw/calibracion.FIT')
fig = calibration.plot((540, 10), None)
fig.savefig("Calibration.png")
plt.close()

In [5]:
files = glob('raw/*.FIT')
for i in range(len(files)):
    if "calibracion" in files[i]:
        del files[i]
        break

In [6]:
spectrums = [Spectrum(item) for item in files]
for spectrum in spectrums:
    fig = spectrum.plot((244, 14), (350, 440))
    fig.savefig("%s.png"%spectrum.name)
    plt.close()